# Toxic App Main

## Exploratory Analysis (Xuyang)

In [4]:
#[Link to Toxic_App_Exploratory_Data_Cleaning]

## Pre-processing of Data (Xuyang)

In [5]:
#[Link to Toxic_App_Exploratory_Data_Cleaning]

## Data Transformation (Lisa)

In [6]:
#[Link to Toxic_App_Transformation notebook (if needed)]

### Term Frequencies (Jeff)

### TF*IDF (Jeff)

### Doc2Vec (Lisa)

## Unsupervised Modeling for Exploratory Analysis
### Kmeans (Xuyang)

## Supervised Modeling & Hyperparameter Tuning

### Gradiant Descent/Logistic Regression (Xuyang)

### Rocchio/Nearest Centroid (Lisa)

Explanation goes here.

### Naive Bayes (Lisa)

### KNN (Lisa)

## Evaluation of Models - Model Comparisons

In [7]:
#[Dataframe of evaluation]

### Top 3 Models with Unbalanced Dataset using Ensemble